### Example Exploratory Notebook

Use this notebook to explore the data generated by the pipeline in your preferred programming language.

**Note**: This notebook is not executed as part of the pipeline.

In [0]:
%pip install yfinance

In [0]:
import yfinance as yf

NIFTY_50_TICKERS = [
    "ADANIENT.NS", "ADANIPORTS.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS",
    "AXISBANK.NS", "BAJAJ-AUTO.NS", "BAJFINANCE.NS", "BAJAJFINSV.NS",
    "BEL.NS", "BHARTIARTL.NS", "BPCL.NS", "BRITANNIA.NS",
    "CIPLA.NS", "COALINDIA.NS", "DIVISLAB.NS", "DRREDDY.NS",
    "EICHERMOT.NS", "GRASIM.NS", "HCLTECH.NS", "HDFCBANK.NS",
    "HDFCLIFE.NS", "HEROMOTOCO.NS", "HINDALCO.NS", "HINDUNILVR.NS",
    "ICICIBANK.NS", "ITC.NS", "INDUSINDBK.NS", "INFY.NS",
    "JSWSTEEL.NS", "KOTAKBANK.NS", "LT.NS", "M&M.NS",
    "MARUTI.NS", "NESTLEIND.NS", "NTPC.NS", "ONGC.NS",
    "POWERGRID.NS", "RELIANCE.NS", "SBILIFE.NS", "SBIN.NS",
    "SUNPHARMA.NS", "TATACONSUM.NS", "TATAMOTORS.NS", "TATASTEEL.NS",
    "TECHM.NS", "TITAN.NS", "ULTRACEMCO.NS", "WIPRO.NS"
]

import pandas as pd
from datetime import datetime,timedelta

today = datetime.today().strftime('%Y-%m-%d')
yesterday = datetime.today() - timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')


dfs = pd.DataFrame(columns=["Close", "High", "Open", "Low", "Volume", "stock_name", "date"])
for ticker in NIFTY_50_TICKERS:
    data = yf.download(
        ticker,
        start=yesterday,
        end=today,
        interval="1d"
    )[["Close", "High", "Open", "Low", "Volume"]]
    data.columns = [f"col_{i+1}" for i in range(len(data.columns))]
    if data.empty:
        dfs.loc[len(dfs)] = [None, None, None, None, None, ticker, yesterday]
    else:
        dfs.loc[len(dfs)] = [
            data["col_1"].iloc[0],
            data["col_2"].iloc[0],
            data["col_3"].iloc[0],
            data["col_4"].iloc[0],
            data["col_5"].iloc[0],
            ticker,
            yesterday
        ]



In [0]:
display(dfs)

In [0]:
# SparkSession is already available in Databricks notebooks as 'spark'.
# No need to create or import SparkSession.

In [0]:

spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.NSE_SET")
spark.sql("""
    CREATE TABLE IF NOT EXISTS workspace.NSE_SET.NIFTY50_DAILY (
        Close        DOUBLE,
        High         DOUBLE,
        Open         DOUBLE,
        Low          DOUBLE,
        stock_Volume BIGINT,
        stock_name   STRING,
        trade_date   DATE
    )
""")

In [0]:
display(dfs)

In [0]:
spark_df = spark.createDataFrame(dfs)
spark_df = spark_df.withColumnRenamed("Volume", "stock_Volume").withColumnRenamed("date", "trade_date")
spark_df.write.insertInto("workspace.NSE_SET.NIFTY50_DAILY", overwrite=False)

In [0]:
%sql
select * from workspace.NSE_SET.NIFTY50_DAILY